In [ ]:
# APG Algorithm with Modular Components

This notebook trains a neural net to output the optimal policy of a nonlinear RBC model using the Analytical Policy Gradient (APG) algorithm. The code has been organized into modular components.


In [ ]:
# Standard imports
import jax
import flax
import optax
import os
import json
from jax import numpy as jnp, random
from flax import linen as nn
from flax.training import train_state as ts_class, checkpoints
from time import time

# Import our modular components
from neural_nets.neural_nets import ActorCritic
from environemnts.RbcMultiSector import RbcMultiSector
from algorithm.epoch_train import get_apg_train_fn
from algorithm.eval import get_eval_fn
from utilities.plot_results import plot_results

print("JAX devices:", jax.devices())
print("JAX version:", jax.__version__)
print("Flax version:", flax.__version__)
print("Optax version:", optax.__version__)


In [ ]:
## Test Environment and Neural Network

In [ ]:
env = RbcMultiSector(N=8)
rng_test = random.PRNGKey(2)
obs_init, state_init = env.reset(rng_test)

# Test neural network
nn_test = ActorCritic(
    actions_dim=env.action_dim,
    hidden_dims_actor=[16, 8],
    hidden_dims_critic=[16, 8],
    activation_final_actor=nn.softmax
)
params_test = nn_test.init(rng_test, obs_init)
action_test, value_test = nn_test.apply(params_test, obs_init)

# Test environment step
new_obs, new_state, reward, done, info = env.step(rng_test, state_init, action_test)
print("Environment step test:")
print(f"New obs shape: {new_obs.shape}")
print(f"Reward: {reward}")
print(f"Done: {done}")
print(f"Action shape: {action_test.shape}")
print(f"Value: {value_test}")

In [ ]:
## Configuration


In [ ]:
# CREATE LEARNING RATE SCHEDULE
lr_schedule = optax.join_schedules(
    schedules=[
        optax.linear_schedule(0, 0.01, 100),
        optax.constant_schedule(0.01),
        optax.constant_schedule(0.001),
        optax.constant_schedule(0.0001),
        optax.cosine_decay_schedule(0.0001, 1000)
    ],
    boundaries=[300, 1000, 1500, 2000]
)

config_apg = {
    "learning_rate": lr_schedule,
    "n_epochs": 60,
    "steps_per_epoch": 50,
    "epis_per_step": 1024*8,
    "periods_per_epis": 32,
    "eval_n_epis": 1024*32,
    "eval_periods_per_epis": 32,
    "gae_lambda": 0.95,
    "max_grad_norm": None,
    "layers_actor": [16, 8],
    "layers_critic": [8, 4],
    "seed": 42,
    "fp64_precision": False,
    "run_name": "apg_RbcMS_modular",
    "date": "modular_implementation",
    "working_dir": "./results/"
}

print("Configuration loaded successfully")


In [ ]:
## Main Experiment Function


In [ ]:
def run_experiment(env, config):
    """Runs experiment."""
    
    print("Starting experiment...\n")
    if config["fp64_precision"]:
        from jax.config import config as config_jax
        config_jax.update("jax_enable_x64", True)
    
    n_cores = len(jax.devices())
    
    # CREATE NN, RNGS, TRAIN_STATE AND EPOCH UPDATE
    nn_policy = ActorCritic(
        actions_dim=env.action_dim,
        hidden_dims_actor=config["layers_actor"],
        hidden_dims_critic=config["layers_critic"],
        activation_final_actor=nn.softmax
    )
    
    if config["max_grad_norm"]:
        optim = optax.chain(
            optax.clip_by_global_norm(config["max_grad_norm"]),
            optax.adam(config["learning_rate"])
        )
    else:
        optim = optax.chain(optax.adam(config["learning_rate"]))
    
    print("Neural Net and Optimizer Created...\n")
    
    # INITIALIZE ENV AND ALGO STATES
    rng, rng_pol, rng_env, rng_epoch, rng_eval = random.split(
        random.PRNGKey(config["seed"]), num=5
    )
    
    obs, env_state = env.reset(rng_env)
    train_state = ts_class.TrainState.create(
        apply_fn=nn_policy.apply,
        params=nn_policy.init(rng_pol, obs),
        tx=optim
    )
    
    # GET EPOCH TRAIN AND EVAL FUNCTIONS
    epoch_update = jax.jit(get_apg_train_fn(env, config))
    eval_fn = jax.jit(get_eval_fn(env, config))
    
    # COMPILE CODE
    print("Starting compilation...\n")
    time_start = time()
    epoch_update(train_state, rng_epoch)  # compiles
    eval_fn(train_state, rng_eval)
    time_compilation = time() - time_start
    print("Time Elapsed for Compilation:", time_compilation, "seconds")
    
    print("Compilation completed. Proceeding to run an epoch and calculate performance statistics...\n")
    
    # RUN AN EPOCH TO GET TIME STATS
    time_start = time()
    epoch_update(train_state, rng_epoch)
    time_epoch = time() - time_start
    print("Time Elapsed for Epoch:", time_epoch, "seconds")
    print("Steps per second:", n_cores*config["steps_per_epoch"]*config["epis_per_step"]*config["periods_per_epis"]/time_epoch, "st/s")
    
    # RUN AN EVAL TO GET TIME STATS
    time_start = time()
    eval_fn(train_state, rng_eval)
    time_eval = time() - time_start
    print("Time Elapsed for Eval:", time_eval, "seconds")
    
    print("Estimated time for full experiment", (time_epoch+time_eval)*config["n_epochs"]/60, "minutes\n")
    
    print("Proceeding to run all epochs...\n")
    
    # CREATE LISTS TO STORE METRICS
    mean_losses, mean_actor_losses, mean_critic_losses, mean_critic_accs, mean_grads, max_grads = [], [], [], [], [], []
    
    # RUN ALL THE EPOCHS
    time_start = time()
    for i in range(1, config["n_epochs"]+1):
        train_state, rng_epoch, epoch_metrics = epoch_update(train_state, rng_epoch)
        eval_metrics = eval_fn(train_state, rng_eval)
        
        mean_losses.append(float(jnp.mean(epoch_metrics[0][0])))
        mean_actor_losses.append(float(jnp.mean(epoch_metrics[0][1][0])))
        mean_critic_losses.append(float(jnp.mean(epoch_metrics[0][1][1])))
        mean_critic_accs.append(float((1-jnp.abs(jnp.mean(epoch_metrics[0][1][2])))*100))
        mean_grads.append(float(jnp.mean(epoch_metrics[1][0])))
        max_grads.append(float(jnp.mean(jnp.max(epoch_metrics[1][1]))))
        
        print('Iteration:', i*config["steps_per_epoch"],
              ", Mean_loss:", jnp.mean(epoch_metrics[0][0]),
              ", Mean_actor_loss:", jnp.mean(epoch_metrics[0][1][0]),
              ", Mean_critic_loss:", jnp.mean(epoch_metrics[0][1][1]),
              ", Mean_critic_acc:", (1-jnp.abs(jnp.mean(epoch_metrics[0][1][2])))*100,
              ", Mean_grads:", jnp.mean(epoch_metrics[1][0]),
              ", Max_grads:", jnp.max(epoch_metrics[1][1]),
              ", Learning rate:", config["learning_rate"](i*config["steps_per_epoch"]),
              "\n")
        
        print('Evaluation:     ',
              ", Mean_loss:", eval_metrics[0],
              ", Mean_actor_loss:", eval_metrics[1],
              ", Mean_critic_loss:", eval_metrics[2],
              ", Mean_critic_acc:", eval_metrics[3],
              ", Mean_grads:", eval_metrics[4],
              ", Max_grads:", eval_metrics[5],
              "\n")
    
    # STORE RESULTS
    print("Minimum loss attained in training:", min(mean_losses))
    
    time_fullexp = (time() - time_start)/60
    print("Time Elapsed for Full Experiment:", time_fullexp, "minutes")
    
    results = {
        "min_loss": min(mean_losses),
        "min_actor_loss": min(mean_actor_losses),
        "min_critic_loss": min(mean_critic_losses),
        "last_critic_accs": mean_critic_accs[-1],
        "Time for Full Experiment (m)": time_fullexp,
        "Time for epoch (s)": time_epoch,
        "Time for Compilation (s)": time_compilation,
        "Steps per second": n_cores * config["steps_per_epoch"] * config["periods_per_epis"]/time_epoch,
        "n_cores": n_cores,
        "periods_per_epis": config["periods_per_epis"],
        "epis_per_step": config["epis_per_step"],
        "steps_per_epoch": config["steps_per_epoch"],
        "n_epochs": config["n_epochs"],
        "layers_actor": config["layers_actor"],
        "layers_critic": config["layers_critic"],
        "date": config["date"],
        "seed": config["seed"],
        "Losses_list": mean_losses,
        "Actor_losses_list": mean_actor_losses,
        "Critic_losses_list": mean_critic_losses,
        "Critic_accs_list": mean_critic_accs,
        "Mean_grads_list": mean_grads,
        "Max_grads_list": max_grads,
    }
    
    # Create results directory if it doesn't exist
    if not os.path.exists(config['working_dir']):
        os.makedirs(config['working_dir'])
    if not os.path.exists(config['working_dir']+config['run_name']):
        os.mkdir(config['working_dir']+config['run_name'])
    
    # Save results
    with open(config['working_dir']+config['run_name']+"/results.json", "w") as write_file:
        json.dump(results, write_file)
    
    # Store checkpoint
    checkpoints.save_checkpoint(
        ckpt_dir=config['working_dir']+config['run_name'],
        target=train_state,
        step=config["n_epochs"]*config["steps_per_epoch"]
    )
    
    return train_state, results


In [ ]:
## Run Experiment


In [ ]:
# Run the main experiment
final_train_state, results = run_experiment(RbcMultiSector(N=8), config_apg)


In [ ]:
## Plot Results


In [ ]:
# Plot the training results
plot_results(config_apg, results)


In [ ]:
## Test Trained Policy


In [ ]:
# Test policies in steady state
env_test = RbcMultiSector(N=8)
params_test = final_train_state.params
nn_policy_test = final_train_state.apply_fn
rng_test = random.PRNGKey(1)

# Test steady state policies
obs_init, _ = env_test.reset(rng_test)
obs_ss = jnp.zeros_like(obs_init, dtype=jnp.float32)
policy_ss = nn_policy_test(params_test, obs_ss)
print("Trained Policy in steady state (should be ~ 1):")
print(f"Policy: {policy_ss[0]}")
print(f"Value: {policy_ss[1]}")
print(f"Policy sum: {jnp.sum(policy_ss[0])} (should be close to 1)")


In [ ]:
## Summary


In [ ]:
print("\n=== EXPERIMENT SUMMARY ===")
print(f"Minimum loss achieved: {results['min_loss']:.6f}")
print(f"Final critic accuracy: {results['last_critic_accs']:.2f}%")
print(f"Total experiment time: {results['Time for Full Experiment (m)']:.2f} minutes")
print(f"Steps per second: {results['Steps per second']:.0f}")
print(f"Results saved to: {config_apg['working_dir']}{config_apg['run_name']}")
